# tf-idf Calculator

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle as pkl
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
path2root = '.'

## load data

In [9]:
data_df = pkl.load(open(f'{path2root}/data/data_df_filtered.pkl', 'rb'))

In [10]:
data_df

,title_1,ingredients_1,ingredients_2,ingredients_3,instructions_1,instructions_3,energy,fat,protein,salt,saturates,sugars
a0c10a8a8e,Red White & Blue Refresher,"[pineapple, raw, all varieties, strawberries, ...","[pineapple sherbet, strawberries, blueberries,...",[1 quart pineapple sherbet (may substitute lem...,[Divide the sherbert between four dessert cups...,[Divide the sherbert between four dessert cups...,59.840909,0.227727,0.701364,0.003295,0.035432,12.039773
6c7d70b7eb,Green Bean Tex-Mex Casserole,"[beans, snap, green, raw, sauce, salsa, ready-...","[frozen green beans, salsa, tortilla chips, ch...","[3 cups frozen green beans (or fresh), 12 cup ...","[Heat oven to 375F., Thaw green beans if froze...","[Heat oven to 375F., Thaw green beans if froze...",179.872047,10.646654,7.977854,1.607530,5.269193,2.617126
60d587b26b,Peach Pie Without Crust Recipe,"[wheat flour, white, all-purpose, unenriched, ...","[flour, sugar, salt, cinnamon, nutmeg, butter]","[1 1/4 c. flour, 1/2 c. sugar, 1/2 teaspoon sa...","[Mix dry ingredients., Cut in butter with blen...","[Mix dry ingredients., Cut in butter with blen...",445.083424,18.716310,4.993583,0.870520,11.439752,29.939530
7e41c984d6,Cinnamon and Raisin Granola Cookies,"[applesauce, canned, unsweetened, without adde...","[unsweetened applesauce, maple syrup, vanilla ...","[1/2 cups Unsweetened Applesauce, 2 Tablespoon...",[Preheat oven to 350 degrees F. Line baking sh...,[Preheat oven to 350 degrees F. Line baking sh...,288.809393,1.096130,3.704287,0.165925,0.263158,0.473074
419438a7cf,Cheesy Chicken Chowder,"[soup, chicken broth or bouillon, dry, potatoe...","[chicken broth, potatoes, celery, salt, carrot...","[3 cups chicken broth, 2 cups diced potatoes, ...","[Bring chicken broth to a boil., Add potatoes,...","[Bring chicken broth to a boil., Add potatoes,...",373.112046,31.399977,5.082541,2.227225,6.364711,1.981984
...,...,...,...,...,...,...,...,...,...,...,...,...
174be11c63,Autumn Oat Favourites,"[butter, without salt, sugars, brown, syrups, ...","[unsalted butter, light brown sugar, golden sy...","[12 cup unsalted butter, plus additional for t...",[Preheat the oven to 325F Grease two baking sh...,[Preheat the oven to 325F Grease two baking sh...,535.182696,36.288458,6.201449,0.043953,27.267494,12.855228
3bac2c373a,Glazed Chicken,"[chicken, broiler or fryers, breast, skinless,...","[chicken breasts, salt, ground pepper, olive o...","[16 ounces chicken breasts (four 4-oz pieces),...",[Season both sides of chicken with salt and pe...,[Season both sides of chicken with salt and pe...,130.103798,3.531694,17.078733,11.262382,0.694414,3.585407
1724848bd1,Polenta with Broccoli,"[water, bottled, generic, broccoli, raw, salt,...","[water, broccoli florets, salt, cornmeal, marg...","[6-1/2 cup water, 1 cup broccoli florets, 1 ts...","[Bring water to a boil in a large stockpot., A...","[Bring water to a boil in a large stockpot., A...",70.605479,1.453553,1.259982,0.306909,0.254675,0.334075
1edd0ac635,Sausage Gravy I,"[pork, fresh, loin, tenderloin, separable lean...","[pork sausage, bacon grease, all - purpose flo...","[1 pound ground pork sausage, 3 tablespoons ba...",[Brown sausage in a large skillet over medium-...,[Brown sausage in a large skillet over medium-...,80.093728,2.024068,10.111346,0.424884,0.729954,3.058921


## Add text column

In [11]:
f = lambda x: f"""{x['title_1']}\n{"; ".join(x['ingredients_3'])}\n{"; ".join(x['instructions_3'])}"""
data_df['text'] = data_df.apply(f, axis=1)
data_df

,title_1,ingredients_1,ingredients_2,ingredients_3,instructions_1,instructions_3,energy,fat,protein,salt,saturates,sugars,text
a0c10a8a8e,Red White & Blue Refresher,"[pineapple, raw, all varieties, strawberries, ...","[pineapple sherbet, strawberries, blueberries,...",[1 quart pineapple sherbet (may substitute lem...,[Divide the sherbert between four dessert cups...,[Divide the sherbert between four dessert cups...,59.840909,0.227727,0.701364,0.003295,0.035432,12.039773,Red White & Blue Refresher\n1 quart pineapple ...
6c7d70b7eb,Green Bean Tex-Mex Casserole,"[beans, snap, green, raw, sauce, salsa, ready-...","[frozen green beans, salsa, tortilla chips, ch...","[3 cups frozen green beans (or fresh), 12 cup ...","[Heat oven to 375F., Thaw green beans if froze...","[Heat oven to 375F., Thaw green beans if froze...",179.872047,10.646654,7.977854,1.607530,5.269193,2.617126,Green Bean Tex-Mex Casserole\n3 cups frozen gr...
60d587b26b,Peach Pie Without Crust Recipe,"[wheat flour, white, all-purpose, unenriched, ...","[flour, sugar, salt, cinnamon, nutmeg, butter]","[1 1/4 c. flour, 1/2 c. sugar, 1/2 teaspoon sa...","[Mix dry ingredients., Cut in butter with blen...","[Mix dry ingredients., Cut in butter with blen...",445.083424,18.716310,4.993583,0.870520,11.439752,29.939530,Peach Pie Without Crust Recipe\n1 1/4 c. flour...
7e41c984d6,Cinnamon and Raisin Granola Cookies,"[applesauce, canned, unsweetened, without adde...","[unsweetened applesauce, maple syrup, vanilla ...","[1/2 cups Unsweetened Applesauce, 2 Tablespoon...",[Preheat oven to 350 degrees F. Line baking sh...,[Preheat oven to 350 degrees F. Line baking sh...,288.809393,1.096130,3.704287,0.165925,0.263158,0.473074,Cinnamon and Raisin Granola Cookies\n1/2 cups ...
419438a7cf,Cheesy Chicken Chowder,"[soup, chicken broth or bouillon, dry, potatoe...","[chicken broth, potatoes, celery, salt, carrot...","[3 cups chicken broth, 2 cups diced potatoes, ...","[Bring chicken broth to a boil., Add potatoes,...","[Bring chicken broth to a boil., Add potatoes,...",373.112046,31.399977,5.082541,2.227225,6.364711,1.981984,Cheesy Chicken Chowder\n3 cups chicken broth; ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174be11c63,Autumn Oat Favourites,"[butter, without salt, sugars, brown, syrups, ...","[unsalted butter, light brown sugar, golden sy...","[12 cup unsalted butter, plus additional for t...",[Preheat the oven to 325F Grease two baking sh...,[Preheat the oven to 325F Grease two baking sh...,535.182696,36.288458,6.201449,0.043953,27.267494,12.855228,"Autumn Oat Favourites\n12 cup unsalted butter,..."
3bac2c373a,Glazed Chicken,"[chicken, broiler or fryers, breast, skinless,...","[chicken breasts, salt, ground pepper, olive o...","[16 ounces chicken breasts (four 4-oz pieces),...",[Season both sides of chicken with salt and pe...,[Season both sides of chicken with salt and pe...,130.103798,3.531694,17.078733,11.262382,0.694414,3.585407,Glazed Chicken\n16 ounces chicken breasts (fou...
1724848bd1,Polenta with Broccoli,"[water, bottled, generic, broccoli, raw, salt,...","[water, broccoli florets, salt, cornmeal, marg...","[6-1/2 cup water, 1 cup broccoli florets, 1 ts...","[Bring water to a boil in a large stockpot., A...","[Bring water to a boil in a large stockpot., A...",70.605479,1.453553,1.259982,0.306909,0.254675,0.334075,Polenta with Broccoli\n6-1/2 cup water; 1 cup ...
1edd0ac635,Sausage Gravy I,"[pork, fresh, loin, tenderloin, separable lean...","[pork sausage, bacon grease, all - purpose flo...","[1 pound ground pork sausage, 3 tablespoons ba...",[Brown sausage in a large skillet over medium-...,[Brown sausage in a large skillet over medium-...,80.093728,2.024068,10.111346,0.424884,0.729954,3.058921,Sausage Gravy I\n1 pound ground pork sausage; ...


## tf-idf vectorization

In [12]:
vectorizer = TfidfVectorizer(stop_words='english')
tf_idfs = vectorizer.fit_transform(data_df['text'].values)
vectorizer.get_feature_names_out()

array(['00', '04', '05', ..., 'zucchinni', 'zydeco', 'zylitol'],
      dtype=object)

## Calc similarities

In [15]:
tf_idfs = sparse.csr_matrix(tf_idfs)
similarities = cosine_similarity(tf_idfs)
similarities

array([[1.        , 0.03541263, 0.02621738, ..., 0.01229037, 0.00586341,
        0.02157534],
       [0.03541263, 1.        , 0.00700124, ..., 0.02154632, 0.01459842,
        0.02931193],
       [0.02621738, 0.00700124, 1.        , ..., 0.02702681, 0.03632465,
        0.07460742],
       ...,
       [0.01229037, 0.02154632, 0.02702681, ..., 1.        , 0.0887568 ,
        0.01686191],
       [0.00586341, 0.01459842, 0.03632465, ..., 0.0887568 , 1.        ,
        0.01272971],
       [0.02157534, 0.02931193, 0.07460742, ..., 0.01686191, 0.01272971,
        1.        ]])

In [18]:
similarities_df = pd.DataFrame(similarities,index=data_df.index, columns=data_df.index)
similarities_df

,a0c10a8a8e,6c7d70b7eb,60d587b26b,7e41c984d6,419438a7cf,2130f4a93c,ee160b7ed1,172b582de2,1d85365954,0162a6bd5c,...,d85f5e71bb,788c420ba7,05ea70e832,c0ca67aebe,045cbf155e,174be11c63,3bac2c373a,1724848bd1,1edd0ac635,da991dd46a
a0c10a8a8e,1.000000,0.035413,0.026217,0.014202,0.051022,0.011517,0.005737,0.014152,0.031075,0.061042,...,0.007868,0.019565,0.029663,0.019063,0.057461,0.035567,0.005617,0.012290,0.005863,0.021575
6c7d70b7eb,0.035413,1.000000,0.007001,0.031480,0.076984,0.023355,0.039113,0.038409,0.028761,0.017359,...,0.030560,0.034583,0.025012,0.018956,0.018739,0.061224,0.023737,0.021546,0.014598,0.029312
60d587b26b,0.026217,0.007001,1.000000,0.081927,0.017332,0.207157,0.008605,0.043884,0.037566,0.096308,...,0.032897,0.057700,0.032771,0.031193,0.045485,0.070115,0.039574,0.027027,0.036325,0.074607
7e41c984d6,0.014202,0.031480,0.081927,1.000000,0.059339,0.184654,0.068163,0.102089,0.113964,0.027635,...,0.038491,0.061879,0.065827,0.033855,0.049380,0.319063,0.073876,0.032556,0.051875,0.092907
419438a7cf,0.051022,0.076984,0.017332,0.059339,1.000000,0.098044,0.033297,0.056125,0.075210,0.036086,...,0.024760,0.104892,0.055313,0.056337,0.057982,0.064855,0.261114,0.090482,0.064953,0.057464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174be11c63,0.035567,0.061224,0.070115,0.319063,0.064855,0.271727,0.050959,0.061205,0.095004,0.068147,...,0.074420,0.068578,0.024816,0.074241,0.053551,1.000000,0.064713,0.049665,0.107490,0.122026
3bac2c373a,0.005617,0.023737,0.039574,0.073876,0.261114,0.100486,0.178428,0.010146,0.232419,0.066510,...,0.018859,0.169763,0.020962,0.095908,0.097255,0.064713,1.000000,0.061114,0.150152,0.017413
1724848bd1,0.012290,0.021546,0.027027,0.032556,0.090482,0.060427,0.062096,0.018168,0.015272,0.060530,...,0.018287,0.032521,0.022854,0.106004,0.077359,0.049665,0.061114,1.000000,0.088757,0.016862
1edd0ac635,0.005863,0.014598,0.036325,0.051875,0.064953,0.142748,0.157363,0.026358,0.078139,0.030425,...,0.036260,0.044222,0.017714,0.079828,0.058991,0.107490,0.150152,0.088757,1.000000,0.012730


## Save

In [19]:
pkl.dump(similarities_df, open(f'{path2root}/data/tf_idf_similarities.pkl', 'wb'))